In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import holoviews as hv


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Concatenate, Permute, Multiply, Reshape

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

from joblib import dump, load, parallel_backend, Parallel, delayed


from tensorflow.keras.utils import to_categorical, plot_model
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
survey_2010 = pd.read_csv('Resources/cut_surveys/2010_cut.csv')
survey_2011 = pd.read_csv('Resources/cut_surveys/2011_cut.csv')
survey_2012_0 = pd.read_csv('Resources/cut_surveys/2012_cut_0.csv')
survey_2012_1 = pd.read_csv('Resources/cut_surveys/2012_cut_1.csv')
survey_2012_2 = pd.read_csv('Resources/cut_surveys/2012_cut_2.csv')
survey_2012_3 = pd.read_csv('Resources/cut_surveys/2012_cut_3.csv')
survey_2013 = pd.read_csv('Resources/cut_surveys/2013_cut.csv')
survey_2014 = pd.read_csv('Resources/cut_surveys/2014_cut.csv')
survey_2015 = pd.read_csv('Resources/cut_surveys/2015_cut.csv')
survey_2016 = pd.read_csv('Resources/cut_surveys/2016_cut.csv')
survey_2017 = pd.read_csv('Resources/cut_surveys/2017_cut.csv')
survey_2018_0 = pd.read_csv('Resources/cut_surveys/2018_cut_0.csv')
survey_2018_1 = pd.read_csv('Resources/cut_surveys/2018_cut_1.csv')
survey_2018_2 = pd.read_csv('Resources/cut_surveys/2018_cut_2.csv')
survey_2019_0 = pd.read_csv('Resources/cut_surveys/2019_cut_0.csv')
survey_2019_1 = pd.read_csv('Resources/cut_surveys/2019_cut_1.csv')
survey_2019_2 = pd.read_csv('Resources/cut_surveys/2019_cut_2.csv')
survey_2020_0 = pd.read_csv('Resources/cut_surveys/2020_cut_0.csv')
survey_2020_1 = pd.read_csv('Resources/cut_surveys/2020_cut_1.csv')
survey_2020_2 = pd.read_csv('Resources/cut_surveys/2020_cut_2.csv')
survey_2020_3 = pd.read_csv('Resources/cut_surveys/2020_cut_3.csv')
survey_2020_4 = pd.read_csv('Resources/cut_surveys/2020_cut_4.csv')
survey_2021 = pd.read_csv('Resources/cut_surveys/2021_cut.csv')
survey_2022_0 = pd.read_csv('Resources/cut_surveys/2022_cut_0.csv')
survey_2022_1 = pd.read_csv('Resources/cut_surveys/2022_cut_1.csv')
survey_2022_2 = pd.read_csv('Resources/cut_surveys/2022_cut_2.csv')

survey_2012 = pd.concat([survey_2012_0, survey_2012_1, survey_2012_2, survey_2012_3], ignore_index=True)
survey_2018 = pd.concat([survey_2018_0, survey_2018_1, survey_2018_2], ignore_index=True)
survey_2019 = pd.concat([survey_2019_0, survey_2019_1, survey_2019_2], ignore_index=True)
survey_2020 = pd.concat([survey_2020_0, survey_2020_1, survey_2020_2, survey_2020_3, survey_2020_4], ignore_index=True)
survey_2022 = pd.concat([survey_2022_0, survey_2022_1, survey_2022_2], ignore_index=True)

In [ ]:
def scale_to_1_to_5(column_values):
    """
    Scale a list of values to a range of 1 to 5, handling 0 and NaN values.

    Args:
    column_values (list): A list of numerical values to be scaled.

    Returns:
    list: A new list with scaled values in the range of 1 to 5.
    """
    scaled_values = []  # Create an empty list to store scaled values
    min_value = np.nanmin(column_values)  # Find the minimum value in the input column, ignoring NaN
    max_value = np.nanmax(column_values)  # Find the maximum value in the input column, ignoring NaN

    # Check if all values are NaN, in which case return a list of NaN
    if np.isnan(min_value) and np.isnan(max_value):
        return [np.nan] * len(column_values)

    # Iterate through each value in the input column
    for value in column_values:
        # Check if the value is NaN, and if so, append NaN to the result list
        if np.isnan(value):
            scaled_values.append(np.nan)
        else:
            # Scale the non-NaN value to the range of 1 to 5 using a linear transformation
            scaled_value = 1 + ((value - min_value) / (max_value - min_value)) * 4
            scaled_values.append(scaled_value)  # Append the scaled value to the result list

    return scaled_values

In [ ]:
print(survey_2010.shape)
print(survey_2010.columns.unique())
for col in survey_2010.columns:
    print(col, survey_2010[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 78 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2010['DEIA'] = 0

#Add Circumstantial with sets of zero
survey_2010['circumstantial'] = 0

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2010['DLEAVING'].replace('A', 5, inplace=True)
survey_2010['DLEAVING'].replace('B', 4, inplace=True)
survey_2010['DLEAVING'].replace('C', 3, inplace=True)
survey_2010['DLEAVING'].replace('D', 2, inplace=True)
survey_2010['DLEAVING'].replace('E', 1, inplace=True)


survey_2010['DRETIRE'].replace('A', 1, inplace=True)
survey_2010['DRETIRE'].replace('B', 2, inplace=True)
survey_2010['DRETIRE'].replace('C', 3, inplace=True)
survey_2010['DRETIRE'].replace('D', 4, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']


survey_2010.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2010[my_work_experience_columns] = survey_2010[my_work_experience_columns].apply(pd.to_numeric)
survey_2010[my_work_unit_columns] = survey_2010[my_work_unit_columns].apply(pd.to_numeric)
survey_2010[my_organization_columns] = survey_2010[my_organization_columns].apply(pd.to_numeric)
survey_2010[my_supervisor_columns] = survey_2010[my_supervisor_columns].apply(pd.to_numeric)
survey_2010[leadership_columns] = survey_2010[leadership_columns].apply(pd.to_numeric)
survey_2010[my_satisfaction_columns] = survey_2010[my_satisfaction_columns].apply(pd.to_numeric)
survey_2010[employee_experience_columns] = survey_2010[employee_experience_columns].apply(pd.to_numeric)
survey_2010[negativity_index_columns] = survey_2010[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2010.columns:
    # check if numeric
    if survey_2010[col].dtype == 'int64' or survey_2010[col].dtype == 'float64':
        # scale to 1-5
        survey_2010[col] = scale_to_1_to_5(survey_2010[col])




# Average all of the values in the columns and drop the columns
survey_2010['my_work_experience'] = survey_2010[my_work_experience_columns].mean(axis=1)
survey_2010['my_work_unit'] = survey_2010[my_work_unit_columns].mean(axis=1)
survey_2010['my_organization'] = survey_2010[my_organization_columns].mean(axis=1)
survey_2010['my_supervisor'] = survey_2010[my_supervisor_columns].mean(axis=1)
survey_2010['leadership'] = survey_2010[leadership_columns].mean(axis=1)
survey_2010['my_satisfaction'] = survey_2010[my_satisfaction_columns].mean(axis=1)
survey_2010['employee_experience'] = survey_2010[employee_experience_columns].mean(axis=1)
survey_2010['negativity_index'] = survey_2010[negativity_index_columns].mean(axis=1)

excess_columsn = ['USERID', 'SUBELEM', 'DLOC', 'DSUPER', 'DSEX', 'DMINORITY',
       'DAGEGRP', 'DPAYCAT', 'DFEDTEN', 'DAGYTEN', 'DLEAVING', 'DRETIRE',  'POSTWT']

# Drop all of the columns

survey_2010.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

# convert AGENCY to agency
survey_2010.rename(columns={'AGENCY': 'agency'}, inplace=True)




In [ ]:
print(survey_2010.shape)
print(survey_2010.columns.unique())
for col in survey_2010.columns:
    print(col, survey_2010[col].unique())

In [ ]:
print(survey_2011.shape)
print(survey_2011.columns.unique())
for col in survey_2011.columns:
    print(col, survey_2011[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']

# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 84 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
                                'Q80', 'Q81', 'Q82', 'Q83', 'Q84']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2011['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2011['circumstantial'] = 0
#Add DLEAVING/DRETIRE and drop all other demographics
survey_2011['DLEAVING'].replace('A', 5, inplace=True)
survey_2011['DLEAVING'].replace('B', 4, inplace=True)
survey_2011['DLEAVING'].replace('C', 3, inplace=True)
survey_2011['DLEAVING'].replace('D', 2, inplace=True)
survey_2011['DLEAVING'].replace('E', 1, inplace=True)

survey_2011['DRETIRE'].replace('A', 1, inplace=True)
survey_2011['DRETIRE'].replace('B', 2, inplace=True)
survey_2011['DRETIRE'].replace('C', 3, inplace=True)
survey_2011['DRETIRE'].replace('D', 4, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']


survey_2011.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2011[my_work_experience_columns] = survey_2011[my_work_experience_columns].apply(pd.to_numeric)
survey_2011[my_work_unit_columns] = survey_2011[my_work_unit_columns].apply(pd.to_numeric)
survey_2011[my_organization_columns] = survey_2011[my_organization_columns].apply(pd.to_numeric)
survey_2011[my_supervisor_columns] = survey_2011[my_supervisor_columns].apply(pd.to_numeric)
survey_2011[leadership_columns] = survey_2011[leadership_columns].apply(pd.to_numeric)
survey_2011[my_satisfaction_columns] = survey_2011[my_satisfaction_columns].apply(pd.to_numeric)
survey_2011[employee_experience_columns] = survey_2011[employee_experience_columns].apply(pd.to_numeric)
survey_2011[negativity_index_columns] = survey_2011[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2011.columns:
    # check if numeric
    if survey_2011[col].dtype == 'int64' or survey_2011[col].dtype == 'float64':
        # scale to 1-5
        survey_2011[col] = scale_to_1_to_5(survey_2011[col])

# Average all of the values in the columns and drop the columns
survey_2011['my_work_experience'] = survey_2011[my_work_experience_columns].mean(axis=1)
survey_2011['my_work_unit'] = survey_2011[my_work_unit_columns].mean(axis=1)
survey_2011['my_organization'] = survey_2011[my_organization_columns].mean(axis=1)
survey_2011['my_supervisor'] = survey_2011[my_supervisor_columns].mean(axis=1)
survey_2011['leadership'] = survey_2011[leadership_columns].mean(axis=1)
survey_2011['my_satisfaction'] = survey_2011[my_satisfaction_columns].mean(axis=1)
survey_2011['employee_experience'] = survey_2011[employee_experience_columns].mean(axis=1)
survey_2011['negativity_index'] = survey_2011[negativity_index_columns].mean(axis=1)

excess_columsn = ['USERID', 'SUBELEM', 'DLOC', 'DSUPER', 'DSEX', 'DMINORITY',
       'DAGEGRP', 'DPAYCAT', 'DFEDTEN', 'DAGYTEN', 'DLEAVING', 'DRETIRE',  'POSTWT']

# Drop all of the columns
survey_2011.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

# convert AGENCY to agency
survey_2011.rename(columns={'AGENCY': 'agency'}, inplace=True)

In [ ]:
print(survey_2011.shape)
print(survey_2011.columns.unique())

In [ ]:
print(survey_2012.shape)
print(survey_2012.columns.unique())
for col in survey_2012.columns:
    print(col, survey_2012[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 84 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
                                'Q80', 'Q81', 'Q82', 'Q83', 'Q84']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2012['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2012['circumstantial'] = 0
#Add DLEAVING/DRETIRE and drop all other demographics
survey_2012['DLEAVING'].replace('A', 5, inplace=True)
survey_2012['DLEAVING'].replace('B', 4, inplace=True)
survey_2012['DLEAVING'].replace('C', 3, inplace=True)
survey_2012['DLEAVING'].replace('D', 2, inplace=True)

survey_2012['DRETIRE'].replace('A', 1, inplace=True)
survey_2012['DRETIRE'].replace('B', 5, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']


survey_2012.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2012[my_work_experience_columns] = survey_2012[my_work_experience_columns].apply(pd.to_numeric)
survey_2012[my_work_unit_columns] = survey_2012[my_work_unit_columns].apply(pd.to_numeric)
survey_2012[my_organization_columns] = survey_2012[my_organization_columns].apply(pd.to_numeric)
survey_2012[my_supervisor_columns] = survey_2012[my_supervisor_columns].apply(pd.to_numeric)
survey_2012[leadership_columns] = survey_2012[leadership_columns].apply(pd.to_numeric)
survey_2012[my_satisfaction_columns] = survey_2012[my_satisfaction_columns].apply(pd.to_numeric)
survey_2012[employee_experience_columns] = survey_2012[employee_experience_columns].apply(pd.to_numeric)
survey_2012[negativity_index_columns] = survey_2012[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2012.columns:
    # check if numeric
    if survey_2012[col].dtype == 'int64' or survey_2012[col].dtype == 'float64':
        # scale to 1-5
        survey_2012[col] = scale_to_1_to_5(survey_2012[col])

# Average all of the values in the columns and drop the columns
survey_2012['my_work_experience'] = survey_2012[my_work_experience_columns].mean(axis=1)
survey_2012['my_work_unit'] = survey_2012[my_work_unit_columns].mean(axis=1)
survey_2012['my_organization'] = survey_2012[my_organization_columns].mean(axis=1)
survey_2012['my_supervisor'] = survey_2012[my_supervisor_columns].mean(axis=1)
survey_2012['leadership'] = survey_2012[leadership_columns].mean(axis=1)
survey_2012['my_satisfaction'] = survey_2012[my_satisfaction_columns].mean(axis=1)
survey_2012['employee_experience'] = survey_2012[employee_experience_columns].mean(axis=1)
survey_2012['negativity_index'] = survey_2012[negativity_index_columns].mean(axis=1)

excess_columsn = ['UserID', 'SUBELEM', 'DSUPER', 'DSEX', 'DMINORITY',
       'DAGEGRP', 'DFEDTEN', 'DLEAVING', 'DRETIRE', 'DMIL', 'DDIS','POSTWT']

# Drop all of the columns
survey_2012.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

print(survey_2012.shape)
print(survey_2012.columns.unique())

#convert AGENCY to agency
survey_2012.rename(columns={'AGENCY': 'agency'}, inplace=True)

In [ ]:
print(survey_2013.shape)
print(survey_2013.columns.unique())
for col in survey_2013.columns:
    print(col, survey_2013[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 84 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
                                'Q80', 'Q81', 'Q82', 'Q83', 'Q84']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2013['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2013['circumstantial'] = 0
#Add DLEAVING/DRETIRE and drop all other demographics
survey_2013['DLEAVING'].replace('A', 5, inplace=True)
survey_2013['DLEAVING'].replace('B', 4, inplace=True)
survey_2013['DLEAVING'].replace('C', 3, inplace=True)
survey_2013['DLEAVING'].replace('D', 2, inplace=True)

survey_2013['DRETIRE'].replace('A', 1, inplace=True)
survey_2013['DRETIRE'].replace('B', 5, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']


survey_2013.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2013[my_work_experience_columns] = survey_2013[my_work_experience_columns].apply(pd.to_numeric)
survey_2013[my_work_unit_columns] = survey_2013[my_work_unit_columns].apply(pd.to_numeric)
survey_2013[my_organization_columns] = survey_2013[my_organization_columns].apply(pd.to_numeric)
survey_2013[my_supervisor_columns] = survey_2013[my_supervisor_columns].apply(pd.to_numeric)
survey_2013[leadership_columns] = survey_2013[leadership_columns].apply(pd.to_numeric)
survey_2013[my_satisfaction_columns] = survey_2013[my_satisfaction_columns].apply(pd.to_numeric)
survey_2013[employee_experience_columns] = survey_2013[employee_experience_columns].apply(pd.to_numeric)
survey_2013[negativity_index_columns] = survey_2013[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2013.columns:
    # check if numeric
    if survey_2013[col].dtype == 'int64' or survey_2013[col].dtype == 'float64':
        # scale to 1-5
        survey_2013[col] = scale_to_1_to_5(survey_2013[col])

# Average all of the values in the columns and drop the columns
survey_2013['my_work_experience'] = survey_2013[my_work_experience_columns].mean(axis=1)
survey_2013['my_work_unit'] = survey_2013[my_work_unit_columns].mean(axis=1)
survey_2013['my_organization'] = survey_2013[my_organization_columns].mean(axis=1)
survey_2013['my_supervisor'] = survey_2013[my_supervisor_columns].mean(axis=1)
survey_2013['leadership'] = survey_2013[leadership_columns].mean(axis=1)
survey_2013['my_satisfaction'] = survey_2013[my_satisfaction_columns].mean(axis=1)
survey_2013['employee_experience'] = survey_2013[employee_experience_columns].mean(axis=1)
survey_2013['negativity_index'] = survey_2013[negativity_index_columns].mean(axis=1)

excess_columsn = ['USERID', 'PLEVEL1', 'PLEVEL2', 'DSUPER', 'DSEX', 'DMINORITY',
       'DAGEGRP', 'DPAYCAT', 'DFEDTEN', 'DLEAVING', 'DRETIRE', 'DMIL', 'DDIS','POSTWT']

# Drop all of the columns
survey_2013.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

print(survey_2013.shape)
print(survey_2013.columns.unique())

In [ ]:
print(survey_2014.shape)
print(survey_2014.columns.unique())
for col in survey_2014.columns:
    print(col, survey_2014[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 84 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
                                'Q80', 'Q81', 'Q82', 'Q83', 'Q84']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2014['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2014['circumstantial'] = 0
#Add DLEAVING/DRETIRE and drop all other demographics
survey_2014['DLEAVING'].replace('A', 5, inplace=True)
survey_2014['DLEAVING'].replace('B', 4, inplace=True)
survey_2014['DLEAVING'].replace('C', 3, inplace=True)
survey_2014['DLEAVING'].replace('D', 2, inplace=True)

survey_2014['DRETIRE'].replace('A', 1, inplace=True)
survey_2014['DRETIRE'].replace('B', 5, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']

survey_2014.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2014[my_work_experience_columns] = survey_2014[my_work_experience_columns].apply(pd.to_numeric)
survey_2014[my_work_unit_columns] = survey_2014[my_work_unit_columns].apply(pd.to_numeric)
survey_2014[my_organization_columns] = survey_2014[my_organization_columns].apply(pd.to_numeric)
survey_2014[my_supervisor_columns] = survey_2014[my_supervisor_columns].apply(pd.to_numeric)
survey_2014[leadership_columns] = survey_2014[leadership_columns].apply(pd.to_numeric)
survey_2014[my_satisfaction_columns] = survey_2014[my_satisfaction_columns].apply(pd.to_numeric)
survey_2014[employee_experience_columns] = survey_2014[employee_experience_columns].apply(pd.to_numeric)
survey_2014[negativity_index_columns] = survey_2014[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2014.columns:
    # check if numeric
    if survey_2014[col].dtype == 'int64' or survey_2014[col].dtype == 'float64':
        # scale to 1-5
        survey_2014[col] = scale_to_1_to_5(survey_2014[col])

# Average all of the values in the columns and drop the columns
survey_2014['my_work_experience'] = survey_2014[my_work_experience_columns].mean(axis=1)
survey_2014['my_work_unit'] = survey_2014[my_work_unit_columns].mean(axis=1)
survey_2014['my_organization'] = survey_2014[my_organization_columns].mean(axis=1)
survey_2014['my_supervisor'] = survey_2014[my_supervisor_columns].mean(axis=1)
survey_2014['leadership'] = survey_2014[leadership_columns].mean(axis=1)
survey_2014['my_satisfaction'] = survey_2014[my_satisfaction_columns].mean(axis=1)
survey_2014['employee_experience'] = survey_2014[employee_experience_columns].mean(axis=1)

survey_2014['negativity_index'] = survey_2014[negativity_index_columns].mean(axis=1)

excess_columsn = ['USERID', 'PLEVEL1', 'PLEVEL2', 'DSUPER', 'DSEX', 'DMINORITY', 'DEDUC',
       'DAGEGRP', 'DPAYCAT', 'DFEDTEN', 'DLEAVING', 'DRETIRE', 'DMIL', 'DDIS','POSTWT']

# Drop all of the columns
survey_2014.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

print(survey_2014.shape)
print(survey_2014.columns.unique())

In [ ]:
print(survey_2015.shape)
print(survey_2015.columns.unique())
for col in survey_2015.columns:
    print(col, survey_2015[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

# Questions 72 - 84 are work_life questions / or special --> (Employee Experience)
employee_experience_columns = ['Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
                                'Q80', 'Q81', 'Q82', 'Q83', 'Q84']


#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2015['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2015['circumstantial'] = 0
#Add DLEAVING/DRETIRE and drop all other demographics
survey_2015['DLEAVING'].replace('A', 5, inplace=True)
survey_2015['DLEAVING'].replace('B', 4, inplace=True)
survey_2015['DLEAVING'].replace('C', 3, inplace=True)
survey_2015['DLEAVING'].replace('D', 2, inplace=True)

survey_2015['DRETIRE'].replace('A', 1, inplace=True)
survey_2015['DRETIRE'].replace('B', 5, inplace=True)

negativity_index_columns = ['DLEAVING', 'DRETIRE']

survey_2015.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2015[my_work_experience_columns] = survey_2015[my_work_experience_columns].apply(pd.to_numeric)
survey_2015[my_work_unit_columns] = survey_2015[my_work_unit_columns].apply(pd.to_numeric)
survey_2015[my_organization_columns] = survey_2015[my_organization_columns].apply(pd.to_numeric)
survey_2015[my_supervisor_columns] = survey_2015[my_supervisor_columns].apply(pd.to_numeric)
survey_2015[leadership_columns] = survey_2015[leadership_columns].apply(pd.to_numeric)
survey_2015[my_satisfaction_columns] = survey_2015[my_satisfaction_columns].apply(pd.to_numeric)
survey_2015[employee_experience_columns] = survey_2015[employee_experience_columns].apply(pd.to_numeric)
survey_2015[negativity_index_columns] = survey_2015[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2015.columns:
    # check if numeric
    if survey_2015[col].dtype == 'int64' or survey_2015[col].dtype == 'float64':
        # scale to 1-5
        survey_2015[col] = scale_to_1_to_5(survey_2015[col])

# Average all of the values in the columns and drop the columns
survey_2015['my_work_experience'] = survey_2015[my_work_experience_columns].mean(axis=1)
survey_2015['my_work_unit'] = survey_2015[my_work_unit_columns].mean(axis=1)
survey_2015['my_organization'] = survey_2015[my_organization_columns].mean(axis=1)
survey_2015['my_supervisor'] = survey_2015[my_supervisor_columns].mean(axis=1)
survey_2015['leadership'] = survey_2015[leadership_columns].mean(axis=1)
survey_2015['my_satisfaction'] = survey_2015[my_satisfaction_columns].mean(axis=1)
survey_2015['employee_experience'] = survey_2015[employee_experience_columns].mean(axis=1)
survey_2015['negativity_index'] = survey_2015[negativity_index_columns].mean(axis=1)

excess_columsn = ['RANDOM', 'PLEVEL1', 'PLEVEL2', 'DSUPER', 'DSEX', 'DMINORITY', 'DEDUC',
       'DAGEGRP', 'DFEDTEN', 'DLEAVING', 'DRETIRE', 'DMIL', 'DDIS','POSTWT']

# Drop all of the columns
survey_2015.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + employee_experience_columns + excess_columsn, inplace=True)

print(survey_2015.shape)
print(survey_2015.columns.unique())

In [ ]:
print(survey_2016.shape)
print(survey_2016.columns.unique())
for col in survey_2016.columns:
    print(col, survey_2016[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']




#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2016['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2016['circumstantial'] = 0

# Add employee experience with sets of zero
survey_2016['employee_experience'] = 0

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2016['DLEAVING'].replace('A', 5, inplace=True)
survey_2016['DLEAVING'].replace('B', 4, inplace=True)
survey_2016['DLEAVING'].replace('C', 3, inplace=True)
survey_2016['DLEAVING'].replace('D', 2, inplace=True)


negativity_index_columns = ['DLEAVING']


survey_2016.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2016[my_work_experience_columns] = survey_2016[my_work_experience_columns].apply(pd.to_numeric)
survey_2016[my_work_unit_columns] = survey_2016[my_work_unit_columns].apply(pd.to_numeric)
survey_2016[my_organization_columns] = survey_2016[my_organization_columns].apply(pd.to_numeric)
survey_2016[my_supervisor_columns] = survey_2016[my_supervisor_columns].apply(pd.to_numeric)
survey_2016[leadership_columns] = survey_2016[leadership_columns].apply(pd.to_numeric)
survey_2016[my_satisfaction_columns] = survey_2016[my_satisfaction_columns].apply(pd.to_numeric)
survey_2016[negativity_index_columns] = survey_2016[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2016.columns:
    # check if numeric
    if survey_2016[col].dtype == 'int64' or survey_2016[col].dtype == 'float64':
        # scale to 1-5
        survey_2016[col] = scale_to_1_to_5(survey_2016[col])

# Average all of the values in the columns and drop the columns
survey_2016['my_work_experience'] = survey_2016[my_work_experience_columns].mean(axis=1)
survey_2016['my_work_unit'] = survey_2016[my_work_unit_columns].mean(axis=1)
survey_2016['my_organization'] = survey_2016[my_organization_columns].mean(axis=1)
survey_2016['my_supervisor'] = survey_2016[my_supervisor_columns].mean(axis=1)
survey_2016['leadership'] = survey_2016[leadership_columns].mean(axis=1)
survey_2016['my_satisfaction'] = survey_2016[my_satisfaction_columns].mean(axis=1)
survey_2016['negativity_index'] = survey_2016[negativity_index_columns].mean(axis=1)

excess_columsn = ['RANDOM', 'plevel1', 'DSUPER', 'DSEX', 'DMINORITY', 
       'DAGEGRP', 'DLEAVING','POSTWT']

# Drop all of the columns
survey_2016.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + excess_columsn, inplace=True)

#convert AGENCY to agency
survey_2016.rename(columns={'AGENCY': 'agency'}, inplace=True)

print(survey_2016.shape)
print(survey_2016.columns.unique())

In [ ]:
print(survey_2017.shape)
print(survey_2017.columns.unique())
for col in survey_2017.columns:
    print(col, survey_2017[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']




#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2017['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2017['circumstantial'] = 0

# Add employee experience with sets of zero
survey_2017['employee_experience'] = 0

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2017['DLEAVING'].replace('A', 5, inplace=True)
survey_2017['DLEAVING'].replace('B', 4, inplace=True)
survey_2017['DLEAVING'].replace('C', 3, inplace=True)
survey_2017['DLEAVING'].replace('D', 2, inplace=True)


negativity_index_columns = ['DLEAVING']


survey_2017.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2017[my_work_experience_columns] = survey_2017[my_work_experience_columns].apply(pd.to_numeric)
survey_2017[my_work_unit_columns] = survey_2017[my_work_unit_columns].apply(pd.to_numeric)
survey_2017[my_organization_columns] = survey_2017[my_organization_columns].apply(pd.to_numeric)
survey_2017[my_supervisor_columns] = survey_2017[my_supervisor_columns].apply(pd.to_numeric)
survey_2017[leadership_columns] = survey_2017[leadership_columns].apply(pd.to_numeric)
survey_2017[my_satisfaction_columns] = survey_2017[my_satisfaction_columns].apply(pd.to_numeric)
survey_2017[negativity_index_columns] = survey_2017[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2017.columns:
    # check if numeric
    if survey_2017[col].dtype == 'int64' or survey_2017[col].dtype == 'float64':
        # scale to 1-5
        survey_2017[col] = scale_to_1_to_5(survey_2017[col])

# Average all of the values in the columns and drop the columns
survey_2017['my_work_experience'] = survey_2017[my_work_experience_columns].mean(axis=1)
survey_2017['my_work_unit'] = survey_2017[my_work_unit_columns].mean(axis=1)
survey_2017['my_organization'] = survey_2017[my_organization_columns].mean(axis=1)
survey_2017['my_supervisor'] = survey_2017[my_supervisor_columns].mean(axis=1)
survey_2017['leadership'] = survey_2017[leadership_columns].mean(axis=1)
survey_2017['my_satisfaction'] = survey_2017[my_satisfaction_columns].mean(axis=1)
survey_2017['negativity_index'] = survey_2017[negativity_index_columns].mean(axis=1)

excess_columns = ['LEVEL1', 'DSUPER','DEDUC','DFEDTEN', 'DSEX', 'DMINORITY', 
       'DLEAVING','POSTWT']

# Drop all of the columns
survey_2017.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + excess_columns, inplace=True)

#convert AGENCY to agency
survey_2017.rename(columns={'AGENCY': 'agency'}, inplace=True)

print(survey_2017.shape)
print(survey_2017.columns.unique())

In [ ]:
print(survey_2018.shape)
print(survey_2018.columns.unique())
for col in survey_2018.columns:
    print(col, survey_2018[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']




#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2018['DEIA'] = 0
#Add Circumstantial with sets of zero
survey_2018['circumstantial'] = 0

# Add employee experience with sets of zero
survey_2018['employee_experience'] = 0

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2018['DLEAVING'].replace('A', 5, inplace=True)
survey_2018['DLEAVING'].replace('B', 4, inplace=True)
survey_2018['DLEAVING'].replace('C', 3, inplace=True)
survey_2018['DLEAVING'].replace('D', 2, inplace=True)


negativity_index_columns = ['DLEAVING']


survey_2018.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2018[my_work_experience_columns] = survey_2018[my_work_experience_columns].apply(pd.to_numeric)
survey_2018[my_work_unit_columns] = survey_2018[my_work_unit_columns].apply(pd.to_numeric)
survey_2018[my_organization_columns] = survey_2018[my_organization_columns].apply(pd.to_numeric)
survey_2018[my_supervisor_columns] = survey_2018[my_supervisor_columns].apply(pd.to_numeric)
survey_2018[leadership_columns] = survey_2018[leadership_columns].apply(pd.to_numeric)
survey_2018[my_satisfaction_columns] = survey_2018[my_satisfaction_columns].apply(pd.to_numeric)
survey_2018[negativity_index_columns] = survey_2018[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2018.columns:
    # check if numeric
    if survey_2018[col].dtype == 'int64' or survey_2018[col].dtype == 'float64':
        # scale to 1-5
        survey_2018[col] = scale_to_1_to_5(survey_2018[col])

# Average all of the values in the columns and drop the columns
survey_2018['my_work_experience'] = survey_2018[my_work_experience_columns].mean(axis=1)
survey_2018['my_work_unit'] = survey_2018[my_work_unit_columns].mean(axis=1)
survey_2018['my_organization'] = survey_2018[my_organization_columns].mean(axis=1)
survey_2018['my_supervisor'] = survey_2018[my_supervisor_columns].mean(axis=1)
survey_2018['leadership'] = survey_2018[leadership_columns].mean(axis=1)
survey_2018['my_satisfaction'] = survey_2018[my_satisfaction_columns].mean(axis=1)
survey_2018['negativity_index'] = survey_2018[negativity_index_columns].mean(axis=1)

excess_columns = ['LEVEL1', 'DSUPER','DEDUC','DFEDTEN', 'DSEX', 'DMINORITY', 
       'DLEAVING','POSTWT', 'RANDOM']

# Drop all of the columns
survey_2018.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + excess_columns, inplace=True)

#convert AGENCY to agency
survey_2018.rename(columns={'AGENCY': 'agency'}, inplace=True)

print(survey_2018.shape)
print(survey_2018.columns.unique())

In [ ]:
print(survey_2019.shape)
print(survey_2019.columns.unique())
for col in survey_2019.columns:
    print(col, survey_2019[col].unique())


# Questions 1 - 19 are my_work_experience questions
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15',
                                'Q16', 'Q17', 'Q18', 'Q19']


# Questions 20 - 28 are my_work_unit questions
my_work_unit_columns = ['Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
                        'Q28']

# Questions 29 - 41 are my_agency questions --> (My Organization)
my_organization_columns = ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
                            'Q37', 'Q38', 'Q39', 'Q40', 'Q41']

# Questions 42 - 62 are my_work_experience questions
    #Questions 42-52 are My Supervisor
my_supervisor_columns = ['Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
                        'Q50', 'Q51', 'Q52']
    #Questions 53-62 are Leadership
leadership_columns = ['Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60',
                        'Q61', 'Q62']

# Questions 63 - 71 are my_satisfaction questions
my_satisfaction_columns = ['Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
                            'Q71']

circumstantial_columns = ['Q73', 'Q74', 'Q75_1', 'Q75_2', 'Q75_3', 'Q75_4', 'Q75_5',
       'Q75_6', 'Q75_7', 'Q75_8', 'Q75_9', 'Q75_10', 'Q76', 'Q77']




#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2019['DEIA'] = 0

# Add employee experience with sets of zero
survey_2019['employee_experience'] = 0

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2019['DLEAVING'].replace('A', 5, inplace=True)
survey_2019['DLEAVING'].replace('B', 4, inplace=True)
survey_2019['DLEAVING'].replace('C', 3, inplace=True)
survey_2019['DLEAVING'].replace('D', 2, inplace=True)


negativity_index_columns = ['DLEAVING']


survey_2019.replace('X', np.nan, inplace=True)

# convert all values to numeric
survey_2019[my_work_experience_columns] = survey_2019[my_work_experience_columns].apply(pd.to_numeric)
survey_2019[my_work_unit_columns] = survey_2019[my_work_unit_columns].apply(pd.to_numeric)
survey_2019[my_organization_columns] = survey_2019[my_organization_columns].apply(pd.to_numeric)
survey_2019[my_supervisor_columns] = survey_2019[my_supervisor_columns].apply(pd.to_numeric)
survey_2019[leadership_columns] = survey_2019[leadership_columns].apply(pd.to_numeric)
survey_2019[my_satisfaction_columns] = survey_2019[my_satisfaction_columns].apply(pd.to_numeric)
survey_2019[circumstantial_columns] = survey_2019[circumstantial_columns].apply(pd.to_numeric)
survey_2019[negativity_index_columns] = survey_2019[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2019.columns:
    # check if numeric
    if survey_2019[col].dtype == 'int64' or survey_2019[col].dtype == 'float64':
        # scale to 1-5
        survey_2019[col] = scale_to_1_to_5(survey_2019[col])

# Average all of the values in the columns and drop the columns
survey_2019['my_work_experience'] = survey_2019[my_work_experience_columns].mean(axis=1)
survey_2019['my_work_unit'] = survey_2019[my_work_unit_columns].mean(axis=1)
survey_2019['my_organization'] = survey_2019[my_organization_columns].mean(axis=1)
survey_2019['my_supervisor'] = survey_2019[my_supervisor_columns].mean(axis=1)
survey_2019['leadership'] = survey_2019[leadership_columns].mean(axis=1)
survey_2019['my_satisfaction'] = survey_2019[my_satisfaction_columns].mean(axis=1)
survey_2019['circumstantial'] = survey_2019[circumstantial_columns].mean(axis=1)
survey_2019['negativity_index'] = survey_2019[negativity_index_columns].mean(axis=1)

excess_columns = ['LEVEL1', 'DSUPER','DEDUC','DFEDTEN', 'DSEX', 'DMINORITY', 
       'DLEAVING','POSTWT', 'RANDOM']

# Drop all of the columns
survey_2019.drop(columns=my_work_experience_columns + my_work_unit_columns + my_organization_columns
                 +  my_supervisor_columns + leadership_columns + my_satisfaction_columns 
                 + excess_columns + circumstantial_columns, inplace=True)

#convert AGENCY to agency
survey_2019.rename(columns={'AGENCY': 'agency'}, inplace=True)

print(survey_2019.shape)
print(survey_2019.columns.unique())


In [ ]:
for col in survey_2019.columns:
    print(col, survey_2019[col].unique())

In [ ]:
print(f"Shape of Data: {survey_2020.shape}")
print(survey_2020.columns.unique())
for column in survey_2020.columns:
    print(f"{column}: {survey_2020[column].unique()}")

#Year 2020 Question Bins

#Q1-Q8= “My Work Experience” 
# (Q1-Q4 has 1-5, after Q5 reponse options include 6th option 'X')

my_work_experience_columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8']

#Q9-Q13= “My Work Unit”
# (Q9 only includes 1-5 response options, rest are 6 options)
my_work_unit_columns=['Q9','Q10','Q11','Q12','Q13']

#Q14-Q18= “My Agency” --> (My Organization)
# (Q17 includes only 5 reponse options)
my_organization_columns=['Q14','Q15','Q16','Q17','Q18']

#Q19-Q25= “My Supervisor”
# (Q21-Q21 include 6 reponse options, Q22-Q25 have 6 )
my_supervisor_columns=['Q19','Q20','Q21','Q22','Q23','Q24','Q25']
#Q26-Q32= “Leadership”
# (all questions include 6 response options)
leadership_columns=['Q26','Q27','Q28','Q29','Q30','Q31','Q32']

#Q33-Q38= “My Satisfaction”
# (all questions include 5 reponse options)
my_satisfaction_columns=['Q33','Q34','Q35','Q36','Q37','Q38']

#V1-V5=“Telework” --> (Circumstantial)
##Create group V2 that includes V2A,V2B, V2C
##Create group V3 that includes V3_01, V3_02, V3_03
	##IF V3 THEN skip V4
#V6-V12= “Employee Supports” --> (Circumstantial)
##Create group V6 that includes V6_01-V6_14
#V13-V14= “Work Supports” --> (Circumstantial)
##Create group V13 that includes V13_01-V13_12
#V15-V20=“Work-Effects” --> (Circumstantial)
##Create group V17 that includes V17_01-V17_06
##Create group V18 that includes V18_01-V18_06
#V21-V24= "Covid Questions" --> (Circumstantial)
##Create group V21 that includes V21_01-V21_05
##Create group V22 that includes V22_01-V22_05

circumstantial_columns= ['V1','V2A','V2B','V2C','V3_01','V3_02','V3_03','V4','V5',
                                         'V6_01','V6_02','V6_03','V6_04','V6_05','V6_06','V6_07','V6_08','V6_09','V6_10','V6_11','V6_12','V6_13','V6_14','V7','V8','V9','V10','V11','V12',
                                         'V13_01','V13_02','V13_03','V13_04','V13_05','V13_06','V13_07','V13_08','V13_09','V13_10','V13_11','V13_12', 'V14',
                                         'V15','V16','V17_01','V17_02','V17_03','V17_04','V17_05','V17_06','V18_01','V18_02','V18_03','V18_04','V18_05','V18_06',
                                         'V19', 'V20','V21_01','V21_02','V21_03','V21_04','V21_05','V22_01','V22_02','V22_03','V22_04','V22_05', 'V23','V24']





# Questions 58 - 64 are work_life questions / or special --> (Employee Experience)
##Create group Q59 that includes Q59_01-Q59_06
employee_experience_columns = ['Q58','Q59_01','Q59_02','Q59_03','Q59_04','Q59_05','Q59_06','Q60','Q61','Q62','Q63','Q64']

#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2020['DEIA'] = 0


#Convert DLEAVING/DRETIRE to 1-5 
survey_2020['DLEAVINGB'].replace('A', 5, inplace=True)
survey_2020['DLEAVINGB'].replace('B', 4, inplace=True)
survey_2020['DLEAVINGB'].replace('C', 3, inplace=True)
survey_2020['DLEAVINGB'].replace('D', 2, inplace=True)




#Add DLEAVINGB and drop all other demographics 
#(This year includes 3 questions for DLEAVING 'Did you want to leave before covid?', leave today?, or decision to leave within the next year changed because of covid'
#   so the leaving question chosen was the one asking if someone wants to leave today which is closest to the question in previous years)
negativity_index_columns = ['DLEAVINGB']



# replace all values of x with 0

survey_2020.replace('X', np.nan, inplace=True)
survey_2020.replace('Y', np.nan, inplace=True)
#replace nan with 0
#survey_2020[my_work_experience_columns] = survey_2020[my_work_experience_columns].fillna(0)

# convert all values to numeric
survey_2020[my_work_experience_columns] = survey_2020[my_work_experience_columns].apply(pd.to_numeric)
survey_2020[my_work_unit_columns] = survey_2020[my_work_unit_columns].apply(pd.to_numeric)
survey_2020[my_organization_columns] = survey_2020[my_organization_columns].apply(pd.to_numeric)
survey_2020[my_supervisor_columns] = survey_2020[my_supervisor_columns].apply(pd.to_numeric)
survey_2020[leadership_columns] = survey_2020[leadership_columns].apply(pd.to_numeric)
survey_2020[my_satisfaction_columns] = survey_2020[my_satisfaction_columns].apply(pd.to_numeric)
survey_2020[employee_experience_columns] = survey_2020[employee_experience_columns].apply(pd.to_numeric)
survey_2020[circumstantial_columns] = survey_2020[circumstantial_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2020.columns:
    # check if numeric
    if survey_2020[col].dtype == 'int64' or survey_2020[col].dtype == 'float64':
        # scale to 1-5
        survey_2020[col] = scale_to_1_to_5(survey_2020[col])

# Average all of the values in the columns and drop the columns
survey_2020['my_work_experience'] = survey_2020[my_work_experience_columns].mean(axis=1)
survey_2020.drop(columns=my_work_experience_columns, inplace=True)

survey_2020['my_work_unit'] = survey_2020[my_work_unit_columns].mean(axis=1)
survey_2020.drop(columns=my_work_unit_columns, inplace=True)


survey_2020['my_organization'] = survey_2020[my_organization_columns].mean(axis=1)
survey_2020.drop(columns=my_organization_columns, inplace=True)

survey_2020['my_supervisor'] = survey_2020[my_supervisor_columns].mean(axis=1)
survey_2020.drop(columns=my_supervisor_columns, inplace=True)

survey_2020['leadership'] = survey_2020[leadership_columns].mean(axis=1)
survey_2020.drop(columns=leadership_columns, inplace=True)

survey_2020['my_satisfaction'] = survey_2020[my_satisfaction_columns].mean(axis=1)
survey_2020.drop(columns=my_satisfaction_columns, inplace=True)

survey_2020['employee_experience'] = survey_2020[employee_experience_columns].mean(axis=1)
survey_2020.drop(columns=employee_experience_columns, inplace=True)

survey_2020['negativity_index'] = survey_2020[negativity_index_columns].mean(axis=1)
survey_2020.drop(columns=negativity_index_columns, inplace=True)

survey_2020['circumstantial'] = survey_2020[circumstantial_columns].mean(axis=1)
survey_2020.drop(columns=circumstantial_columns, inplace=True)

excess_columns= ['RandomID','DRNO', 'DHISP','DDIS','DAGEGRP', 'DSUPER', 'DFEDTEN', 
       'DLEAVINGA','DLEAVINGC','DMIL', 'DSEX', 'POSTWT']
survey_2020.drop(columns=excess_columns, inplace=True)

#Review the dataframe with averaged categories
print(survey_2020.shape)
print(survey_2020.columns.unique())



In [ ]:
print(f"Shape of Data: {survey_2021.shape}")
print(survey_2021.columns.unique())
for column in survey_2021.columns:
    print(f"{column}: {survey_2021[column].unique()}")

# Questions 1- 8 are my_work_experience questions
my_work_experience_columns = ['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8']

# Questions 9 - 19 are my_work_unit questions
my_work_unit_columns = ['Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16','Q17','Q18','Q19']

# Questions 20 - 24 are my_organization questions
my_organization_columns = ['Q20','Q21','Q22','Q23','Q24']

# Questions 25 - 31 and 51 - 53 are my_supervisor questions
my_supervisor_columns = ['Q25','Q26','Q27','Q28','Q29','Q30','Q31','Q51','Q52','Q53']

# Questions 32 - 38, 48 - 50, and 55 are leadership questions
leadership_columns = ['Q32','Q33','Q34','Q35','Q36','Q37','Q38','Q48','Q49','Q50','Q55']

# Questions 39 - 44 are my_satisfaction questions
my_satisfaction_columns = ['Q39','Q40','Q41','Q42','Q43','Q44']

# Questions 45 - 47, 54, 56, and 57 are Pandemic, Transition to the Worksite, Workplace Flexibilities questions (Circumstantial)
circumstantial_columns = ['Q45','Q46','Q47_01',
       'Q47_02', 'Q47_03', 'Q47_04', 'Q47_05', 'Q47_06', 'Q47_07', 'Q47_08',
       'Q47_09', 'Q47_10', 'Q47_11','Q54','Q56','Q57']

#Add Employee Experience with sets of zero
survey_2021['employee_experience'] = 0
#Add Diversity, Equity, Inclusion, and Accessibility with sets of zero
survey_2021['DEIA'] = 0
#Add DLEAVING and drop all other demographics
survey_2021['DLEAVING'].replace('A', 5, inplace=True)
survey_2021['DLEAVING'].replace('B', 4, inplace=True)
survey_2021['DLEAVING'].replace('C', 3, inplace=True)
survey_2021['DLEAVING'].replace('D', 2, inplace=True)

negativity_index_columns = ['DLEAVING']

survey_2021.replace('X', np.nan, inplace=True)
survey_2021.replace('Y', np.nan, inplace=True)

# convert all values to numeric
survey_2021[my_work_experience_columns] = survey_2021[my_work_experience_columns].apply(pd.to_numeric)
survey_2021[my_work_unit_columns] = survey_2021[my_work_unit_columns].apply(pd.to_numeric)
survey_2021[my_organization_columns] = survey_2021[my_organization_columns].apply(pd.to_numeric)
survey_2021[my_supervisor_columns] = survey_2021[my_supervisor_columns].apply(pd.to_numeric)
survey_2021[leadership_columns] = survey_2021[leadership_columns].apply(pd.to_numeric)
survey_2021[my_satisfaction_columns] = survey_2021[my_satisfaction_columns].apply(pd.to_numeric)
survey_2021[circumstantial_columns] = survey_2021[circumstantial_columns].apply(pd.to_numeric)
survey_2021[negativity_index_columns] = survey_2021[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2021.columns:
    # check if numeric
    if survey_2021[col].dtype == 'int64' or survey_2021[col].dtype == 'float64':
        # scale to 1-5
        survey_2021[col] = scale_to_1_to_5(survey_2021[col])

# Average all of the values in the columns and drop the columns
survey_2021['my_work_experience'] = survey_2021[my_work_experience_columns].mean(axis=1)
survey_2021.drop(columns=my_work_experience_columns, inplace=True)

survey_2021['my_work_unit'] = survey_2021[my_work_unit_columns].mean(axis=1)
survey_2021.drop(columns=my_work_unit_columns, inplace=True)

survey_2021['my_organization'] = survey_2021[my_organization_columns].mean(axis=1)
survey_2021.drop(columns=my_organization_columns, inplace=True)

survey_2021['my_supervisor'] = survey_2021[my_supervisor_columns].mean(axis=1)
survey_2021.drop(columns=my_supervisor_columns, inplace=True)

survey_2021['leadership'] = survey_2021[leadership_columns].mean(axis=1)
survey_2021.drop(columns=leadership_columns, inplace=True)

survey_2021['my_satisfaction'] = survey_2021[my_satisfaction_columns].mean(axis=1)
survey_2021.drop(columns=my_satisfaction_columns, inplace=True)

survey_2021['circumstantial'] = survey_2021[circumstantial_columns].mean(axis=1)
survey_2021.drop(columns=circumstantial_columns, inplace=True)

survey_2021['negativity_index'] = survey_2021[negativity_index_columns].mean(axis=1)
survey_2021.drop(columns=negativity_index_columns, inplace=True)

excess_columns= ['DRNO', 'DHISP', 'DDIS', 'DAGEGRP',
       'DSUPER', 'DFEDTEN', 'DSEX', 'DMIL', 'POSTWT', 'RandomID']

survey_2021.drop(columns=excess_columns, inplace=True)

#Review the dataframe with averaged categories
print(survey_2021.shape)
print(survey_2021.columns.unique())


In [ ]:
print(f"Shape of Data: {survey_2022.shape}")
print(survey_2022.columns.unique())
for column in survey_2022.columns:
    print(f"{column}: {survey_2022[column].unique()}")

# Question 1-13 My Work Experience
my_work_experience_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
                                'Q9', 'Q10', 'Q11', 'Q12', 'Q13']

# Question 14-34 My Work Unit
my_work_unit_columns = ['Q14', 'Q15_1', 'Q15_2', 'Q15_3', 'Q15_4', 'Q15_5', 'Q15_6',
                 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
                'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29',
                'Q30', 'Q31', 'Q32', 'Q33', 'Q34']

# Question 35-44 My Organization
my_organization_columns = ['Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42',
                    'Q43', 'Q44']

# Question 45-54 My Supervisor
my_supervisor_columns = ['Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52',
                'Q53', 'Q54']

# Question 55-64 Leadership
leadership_columns = ['Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62',
                'Q63', 'Q64']

# Question 65-70 My Satisfaction
my_satisfaction_columns = ['Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70']

# Question 71-84 Diversity, Equity, Inclusion, and Accessibility
DEIA_columns = ['Q71', 'Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79',
        'Q80', 'Q81', 'Q82', 'Q83', 'Q84']

# Question 85-89 Employee Experience
employee_experience_columns = ['Q85', 'Q86', 'Q87', 'Q88', 'Q89']

# Question 90-99 Pandemic, Transition to the Worksite, Workplace Flexibilities (Circumstantial)
circumstantial_columns = ['Q90', 'Q91', 'Q92', 'Q93', 'Q94', 'Q95', 'Q96', 'Q97',
                'Q98', 'Q99']

#Add DLEAVING
negativity_index_columns = ['DLEAVING']

#Add DLEAVING/DRETIRE and drop all other demographics
survey_2022['DLEAVING'].replace('A', 5, inplace=True)
survey_2022['DLEAVING'].replace('B', 4, inplace=True)
survey_2022['DLEAVING'].replace('C', 3, inplace=True)
survey_2022['DLEAVING'].replace('D', 2, inplace=True)

# replace all x and y values with 0
survey_2022.replace('X', np.nan, inplace=True)
survey_2022.replace('Y', np.nan, inplace=True)

# convert all values to numeric
survey_2022[my_work_experience_columns] = survey_2022[my_work_experience_columns].apply(pd.to_numeric)
survey_2022[my_work_unit_columns] = survey_2022[my_work_unit_columns].apply(pd.to_numeric)
survey_2022[my_organization_columns] = survey_2022[my_organization_columns].apply(pd.to_numeric)
survey_2022[my_supervisor_columns] = survey_2022[my_supervisor_columns].apply(pd.to_numeric)
survey_2022[leadership_columns] = survey_2022[leadership_columns].apply(pd.to_numeric)
survey_2022[my_satisfaction_columns] = survey_2022[my_satisfaction_columns].apply(pd.to_numeric)
survey_2022[DEIA_columns] = survey_2022[DEIA_columns].apply(pd.to_numeric)
survey_2022[employee_experience_columns] = survey_2022[employee_experience_columns].apply(pd.to_numeric)
survey_2022[circumstantial_columns] = survey_2022[circumstantial_columns].apply(pd.to_numeric)
survey_2022[negativity_index_columns] = survey_2022[negativity_index_columns].apply(pd.to_numeric)

#scale all columns to 1-5
for col in survey_2022.columns:
    # check if numeric
    if survey_2022[col].dtype == 'int64' or survey_2022[col].dtype == 'float64':
        # scale to 1-5
        survey_2022[col] = scale_to_1_to_5(survey_2022[col])

# Average all of the values in the columns and drop the columns
survey_2022['my_work_experience'] = survey_2022[my_work_experience_columns].mean(axis=1)
survey_2022.drop(columns=my_work_experience_columns, inplace=True)

survey_2022['my_work_unit'] = survey_2022[my_work_unit_columns].mean(axis=1)
survey_2022.drop(columns=my_work_unit_columns, inplace=True)

survey_2022['my_organization'] = survey_2022[my_organization_columns].mean(axis=1)
survey_2022.drop(columns=my_organization_columns, inplace=True)

survey_2022['my_supervisor'] = survey_2022[my_supervisor_columns].mean(axis=1)
survey_2022.drop(columns=my_supervisor_columns, inplace=True)

survey_2022['leadership'] = survey_2022[leadership_columns].mean(axis=1)
survey_2022.drop(columns=leadership_columns, inplace=True)

survey_2022['my_satisfaction'] = survey_2022[my_satisfaction_columns].mean(axis=1)
survey_2022.drop(columns=my_satisfaction_columns, inplace=True)

survey_2022['DEIA'] = survey_2022[DEIA_columns].mean(axis=1)
survey_2022.drop(columns=DEIA_columns, inplace=True)

survey_2022['employee_experience'] = survey_2022[employee_experience_columns].mean(axis=1)
survey_2022.drop(columns=employee_experience_columns, inplace=True)

survey_2022['circumstantial'] = survey_2022[circumstantial_columns].mean(axis=1)
survey_2022.drop(columns=circumstantial_columns, inplace=True)

survey_2022['negativity_index'] = survey_2022[negativity_index_columns].mean(axis=1)
survey_2022.drop(columns=negativity_index_columns, inplace=True)

excess_columns= ['DRNO', 'DHISP', 'DDIS', 'DAGEGRP', 'DSUPER', 'DFEDTEN', 'DSEX', 'DMIL',
       'POSTWT', 'RandomID']

survey_2022.drop(columns=excess_columns, inplace=True)

print(survey_2022.shape)
print(survey_2022.columns.unique())



In [ ]:
#add each year as a column to each survey
survey_2010['year'] = 2010
survey_2011['year'] = 2011
survey_2012['year'] = 2012
survey_2013['year'] = 2013
survey_2014['year'] = 2014
survey_2015['year'] = 2015
survey_2016['year'] = 2016
survey_2017['year'] = 2017
survey_2018['year'] = 2018
survey_2019['year'] = 2019
survey_2020['year'] = 2020
survey_2021['year'] = 2021
survey_2022['year'] = 2022

# set the column order
column_order = ['year', 'agency', 'my_work_experience', 'my_work_unit', 'my_organization', 'my_supervisor',
                'leadership', 'my_satisfaction', 'employee_experience', 'circumstantial', 'DEIA',
                'negativity_index']

# reorder the columns
survey_2010 = survey_2010[column_order]
survey_2011 = survey_2011[column_order]
survey_2012 = survey_2012[column_order]
survey_2013 = survey_2013[column_order]
survey_2014 = survey_2014[column_order]
survey_2015 = survey_2015[column_order]
survey_2016 = survey_2016[column_order]
survey_2017 = survey_2017[column_order]
survey_2018 = survey_2018[column_order]
survey_2019 = survey_2019[column_order]
survey_2020 = survey_2020[column_order]
survey_2021 = survey_2021[column_order]
survey_2022 = survey_2022[column_order]

In [ ]:
# create a dataframe that compares the columns of each survey

all_year_columns = pd.DataFrame(columns=['2010', '2011', '2012', '2013', '2014', '2015', '2016',
                                            '2017', '2018', '2019', '2020', '2021', '2022'])
for year in all_year_columns.columns:
    # calculate the maximum number of columns
    max_col = max(survey_2010.shape[1], survey_2011.shape[1], survey_2012.shape[1], survey_2013.shape[1],
                  survey_2014.shape[1], survey_2015.shape[1], survey_2016.shape[1], survey_2017.shape[1],
                  survey_2018.shape[1], survey_2019.shape[1], survey_2020.shape[1], survey_2021.shape[1],
                  survey_2022.shape[1])
    # if the number of columns is less than the maximum, add empty columns as a null value
    survey = eval(f'survey_{year}')
    for i in range(max_col - survey.shape[1]):
        survey[f'empty_{i}'] = np.nan
    # add the columns of the survey to the dataframe
    all_year_columns[year] = survey.columns.unique()

    # create a text file that contains all unique column values for each column in each survey, append to the file
    with open('Resources/year_column_values.txt', 'a') as f:
        f.write(f'\n{year}\n')
        for column in survey.columns:
            f.write(f'{column}: {survey[column].unique()}\n')
    
all_year_columns.to_csv('Resources/all_year_columns.csv', index=False)

In [ ]:
# combine all of the surveys into one dataframe

all_surveys = pd.concat([survey_2010, survey_2011, survey_2012, survey_2013, survey_2014, survey_2015,
                            survey_2016, survey_2017, survey_2018, survey_2019, survey_2020, survey_2021,
                            survey_2022], ignore_index=True)




In [ ]:
# use iterative imputer to fill in missing values of DEIA using 2022 as the example dataframe
# create a copy of the dataframe
df = all_surveys.copy()

# create a list of the columns to be used as features
columns_to_use_as_features = ['my_work_experience', 'my_work_unit', 'my_organization', 'my_supervisor',
                              'leadership', 'my_satisfaction', 'employee_experience', 'circumstantial',
                              'negativity_index']

for column in columns_to_use_as_features:
    # fill missing values with 0
    df[column] = df[column].fillna(0)

df['unique_id'] = df.index

# Isolate the data from 2022 which will be used to train the imputer
df_2022 = df[df['year'] == 2022]



# Isolate the rows with missing 'DEIA' values, which we will later impute
df_missing = df[df['year'] != 2022]


#df_missing = df_missing.reset_index(drop=True)

# create a list of the columns to be imputed
columns_to_impute = ['DEIA']











In [ ]:
# Set up the IterativeImputer with a RandomForestRegressor to handle complex relationships
imputer = IterativeImputer(estimator=RandomForestRegressor(n_jobs=-1, warm_start=True, n_estimators=50), max_iter=10, random_state=0)

In [ ]:
# Fit the imputer on 2022 data
imputer.fit(df_2022[columns_to_use_as_features + ['DEIA']])


In [ ]:
dump(imputer, 'Resources/imputer.joblib')

In [ ]:
imputer = load('Resources/imputer.joblib')

In [ ]:
# Impute the missing values
# Make sure to use both the feature columns and the column to be imputed
df_missing_imputed = imputer.transform(df_missing[columns_to_use_as_features + ['DEIA']])


In [ ]:
# Convert the imputed data back to a DataFrame
df_missing_imputed = pd.DataFrame(df_missing_imputed)
print(df_missing_imputed.shape)
print(df_missing.shape)
df_missing_imputed.head()

In [ ]:
print(df_missing[columns_to_use_as_features + ['DEIA']].head())

In [ ]:
# add year column to imputed dataframe from df_missing
df_missing_imputed['year'] = df_missing['year']
df_missing_imputed['agency'] = df_missing['agency']
df_missing_imputed['unique_id'] = df_missing['unique_id']

# rename the columns
df_missing_imputed.rename(columns={0: 'my_work_experience', 1: 'my_work_unit', 2: 'my_organization',
                                   3: 'my_supervisor', 4: 'leadership', 5: 'my_satisfaction',
                                   6: 'employee_experience', 7: 'circumstantial', 8: 'negativity_index',
                                   9: 'DEIA'}, inplace=True)

# reorder the columns
df_missing_imputed = df_missing_imputed[['year', 'agency', 'unique_id', 'my_work_experience', 'my_work_unit',
                                         'my_organization', 'my_supervisor', 'leadership', 'my_satisfaction',
                                         'employee_experience', 'circumstantial', 'DEIA', 'negativity_index']]
print(df_missing_imputed.head())
print(df_missing_imputed.shape)
print(df_missing_imputed.tail())

In [ ]:
# reorder the columns
df_2022 = df_2022[['year', 'agency', 'unique_id', 'my_work_experience', 'my_work_unit',
                                         'my_organization', 'my_supervisor', 'leadership', 'my_satisfaction',
                                         'employee_experience', 'circumstantial', 'DEIA', 'negativity_index']]

print(df_2022.shape)

df_2022.head()

In [ ]:
# Merge the imputed data with the 2022 data
combined_survey = pd.concat([df_missing_imputed, df_2022])
# sort the dataframe by unique_id
combined_survey.sort_values(by='unique_id', inplace=True)

# reset the index
combined_survey.reset_index(drop=True, inplace=True)

# drop the unique_id column
combined_survey.drop(columns='unique_id', inplace=True)

# convert the DEIA column to numeric
combined_survey['DEIA'] = pd.to_numeric(combined_survey['DEIA'])

# drop agencies with DD
combined_survey = combined_survey[combined_survey['agency'] != 'DD']

combined_survey.head()

In [ ]:
# save the imputed dataframe to a csv per year
for year in range(2010, 2023):
    combined_survey[combined_survey['year'] == year].to_csv(f'Resources/survey_imputed/{year}_imputed.csv', index=False)